In [ ]:
from google.colab import drive # Mounting My Drive [Google drive to Collab] so we can access the Healthcare file anytime.
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.listdir('/content/drive/MyDrive/Healthcare_Project') # checking the directory and healthcare folder for CSV file.


['diabetic_data.csv']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/drive/MyDrive/Healthcare_Project/diabetic_data.csv')
df.head()



,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
df.shape

(101766, 50)

In [ ]:
df['readmitted'].value_counts()


,count
readmitted,
NO,54864
>30,35545
<30,11357


In [ ]:
df['readmitted'].value_counts(normalize=True)


,proportion
readmitted,
NO,0.539119
>30,0.349282
<30,0.111599


In [ ]:
(df == "?").sum().sort_values(ascending=False).head(10)


,0
weight,98569
medical_specialty,49949
payer_code,40256
race,2273
diag_3,1423
diag_2,358
diag_1,21
admission_type_id,0
patient_nbr,0
encounter_id,0


In [ ]:
df=df.replace("?",np.nan)
df.isnull().sum().sort_values(ascending=False).head(15)

,0
weight,98569
max_glu_serum,96420
A1Cresult,84748
medical_specialty,49949
payer_code,40256
race,2273
diag_3,1423
diag_2,358
diag_1,21
patient_nbr,0


In [ ]:
#Dropping the columns with High unknown/"?"/Null values
df= df.drop(columns=['weight','max_glu_serum','A1Cresult'])

In [ ]:
#filling the row values with NaN form the below mentioned columns with Unknown word.
categorical_cols = ["medical_specialty", "payer_code", "race",
                    "diag_1", "diag_2", "diag_3"]

for col in categorical_cols:
    df[col] = df[col].fillna("Unknown")


In [ ]:
#Checking if any Null value is Present after changing the Null to "Unknown".
df.isnull().sum().sort_values(ascending=False).head(10)


,0
encounter_id,0
patient_nbr,0
race,0
gender,0
age,0
admission_type_id,0
discharge_disposition_id,0
admission_source_id,0
time_in_hospital,0
payer_code,0


In [ ]:
#We are converting the target variable into Binaries. If Readmitted within 30 days (<30) then 1 or else 0.
df['readmit_30'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

df['readmit_30'].value_counts()
df['readmit_30'].value_counts(normalize=True)


,proportion
readmit_30,
0,0.888401
1,0.111599


In [ ]:
df = df.drop(columns=['encounter_id', 'patient_nbr', 'readmitted'])


In [ ]:
overall_rate = df['readmit_30'].mean()
overall_rate


np.float64(0.11159915885462728)

In [ ]:
age_readmit = df.groupby('age')['readmit_30'].mean().sort_values(ascending=False)
age_readmit


,readmit_30
age,
[20-30),0.142426
[80-90),0.120835
[70-80),0.117731
[30-40),0.112318
[60-70),0.111284
[90-100),0.110992
[40-50),0.106040
[50-60),0.096662
[10-20),0.057887


In [ ]:
df['age'].value_counts()


,count
age,
[70-80),26068
[60-70),22483
[50-60),17256
[80-90),17197
[40-50),9685
[30-40),3775
[90-100),2793
[20-30),1657
[10-20),691


In [ ]:
df.groupby('number_inpatient')['readmit_30'].mean().sort_values(ascending=False).head(10)


,readmit_30
number_inpatient,
17,1.000000
21,1.000000
15,1.000000
11,0.673469
13,0.500000
19,0.500000
12,0.500000
8,0.443709
10,0.426230


In [ ]:
inpatient_summary = df.groupby('number_inpatient')['readmit_30'].agg(['mean','count']).sort_values(by='mean', ascending=False)
inpatient_summary.head(15)


,mean,count
number_inpatient,,
17,1.000000,1
21,1.000000,1
15,1.000000,9
11,0.673469,49
13,0.500000,20
19,0.500000,2
12,0.500000,34
8,0.443709,151
10,0.426230,61


In [ ]:
def inpatient_bucket(x):
    if x == 0:
        return "0 visits"
    elif x <= 2:
        return "1-2 visits"
    elif x <= 5:
        return "3-5 visits"
    elif x <= 10:
        return "6-10 visits"
    else:
        return "11+ visits"

df['inpatient_bucket'] = df['number_inpatient'].apply(inpatient_bucket)

bucket_summary = df.groupby('inpatient_bucket')['readmit_30'].mean().sort_values(ascending=False)
bucket_summary


,readmit_30
inpatient_bucket,
11+ visits,0.586466
6-10 visits,0.374416
3-5 visits,0.227545
1-2 visits,0.141839
0 visits,0.084371


In [ ]:
df['inpatient_bucket'].value_counts()


,count
inpatient_bucket,
0 visits,67630
1-2 visits,27087
3-5 visits,5845
6-10 visits,1071
11+ visits,133


In [ ]:
df.groupby('number_emergency')['readmit_30'].mean().sort_values(ascending=False).head(10)


,readmit_30
number_emergency,
64,1.000000
28,1.000000
20,0.500000
19,0.500000
21,0.500000
22,0.500000
16,0.400000
9,0.363636
10,0.352941


In [ ]:
emergency_summary = df.groupby('number_emergency')['readmit_30'].agg(['mean','count']).sort_values(by='mean', ascending=False)
emergency_summary.head(15)


,mean,count
number_emergency,,
64,1.000000,1
28,1.000000,1
20,0.500000,4
19,0.500000,4
21,0.500000,2
22,0.500000,6
16,0.400000,5
9,0.363636,33
10,0.352941,34


In [ ]:
def emergency_bucket(x):
    if x == 0:
        return "0 visits"
    elif x <= 2:
        return "1-2 visits"
    elif x <= 5:
        return "3-5 visits"
    elif x <= 10:
        return "6-10 visits"
    else:
        return "11+ visits"

df['emergency_bucket'] = df['number_emergency'].apply(emergency_bucket)

emergency_bucket_summary = df.groupby('emergency_bucket')['readmit_30'].mean().sort_values(ascending=False)
emergency_bucket_summary


,readmit_30
emergency_bucket,
6-10 visits,0.285211
11+ visits,0.280899
3-5 visits,0.239349
1-2 visits,0.151765
0 visits,0.104743


In [ ]:
df['emergency_bucket'].value_counts()


,count
emergency_bucket,
0 visits,90383
1-2 visits,9719
3-5 visits,1291
6-10 visits,284
11+ visits,89


In [ ]:
df.groupby('time_in_hospital')['readmit_30'].mean().sort_values(ascending=False).head(10)


,readmit_30
time_in_hospital,
10,0.143467
8,0.142337
9,0.137242
12,0.133287
14,0.129559
7,0.128350
6,0.125879
13,0.123140
5,0.120309


In [ ]:
df.groupby('number_diagnoses')['readmit_30'].mean().sort_values(ascending=False).head(10)



,readmit_30
number_diagnoses,
11,0.272727
15,0.200000
13,0.187500
10,0.176471
14,0.142857
9,0.123802
8,0.118124
12,0.111111
7,0.107669


In [ ]:
df['high_risk_flag'] = (
    (df['number_inpatient'] >= 3) |
    (df['number_emergency'] >= 3) |
    (df['number_diagnoses'] >= 10)
)

df.groupby('high_risk_flag')['readmit_30'].mean()


,readmit_30
high_risk_flag,
False,0.099989
True,0.246467


In [ ]:
df['high_risk_flag'].value_counts()


,count
high_risk_flag,
False,93700
True,8066


In [ ]:
# Cost assumption per readmission (USD)
#Define Cost Assumptions
cost_per_readmission = 12000


In [ ]:
#Total Readmission Cost
total_readmissions = df['readmit_30'].sum()

total_cost = total_readmissions * cost_per_readmission

total_readmissions, total_cost


(np.int64(11357), np.int64(136284000))

In [ ]:
#High-Risk Group Cost
high_risk_df = df[df['high_risk_flag'] == True]

high_risk_readmissions = high_risk_df['readmit_30'].sum()
high_risk_cost = high_risk_readmissions * cost_per_readmission

high_risk_readmissions, high_risk_cost


(np.int64(1988), np.int64(23856000))

In [ ]:
#Multi-Scenario Savings Table
import pandas as pd

reduction_rates = [0.10, 0.20, 0.30]

scenario_results = []

for r in reduction_rates:
    savings = high_risk_cost * r
    scenario_results.append({
        "Reduction Rate": f"{int(r*100)}%",
        "Estimated Savings ($)": savings
    })

scenario_df = pd.DataFrame(scenario_results)

scenario_df


,Reduction Rate,Estimated Savings ($)
0,10%,2385600.0
1,20%,4771200.0
2,30%,7156800.0


In [ ]:
intervention_cost_per_patient = 500

high_risk_patients = len(high_risk_df)

total_intervention_cost = high_risk_patients * intervention_cost_per_patient

roi_results = []

for r in reduction_rates:
    savings = high_risk_cost * r
    net_roi = savings - total_intervention_cost

    roi_results.append({
        "Reduction Rate": f"{int(r*100)}%",
        "Savings ($)": savings,
        "Intervention Cost ($)": total_intervention_cost,
        "Net ROI ($)": net_roi
    })

roi_df = pd.DataFrame(roi_results)

roi_df


,Reduction Rate,Savings ($),Intervention Cost ($),Net ROI ($)
0,10%,2385600.0,4033000,-1647400.0
1,20%,4771200.0,4033000,738200.0
2,30%,7156800.0,4033000,3123800.0


In [ ]:
# Cost per high-risk patient
cost_per_high_risk_patient = high_risk_cost / high_risk_patients

cost_per_high_risk_patient


np.float64(2957.599801636499)

In [ ]:
#Savings per Patient
savings_per_patient_results = []

for r in reduction_rates:
    savings = high_risk_cost * r
    savings_per_patient = savings / high_risk_patients

    savings_per_patient_results.append({
        "Reduction Rate": f"{int(r*100)}%",
        "Savings per Patient ($)": savings_per_patient
    })

savings_per_patient_df = pd.DataFrame(savings_per_patient_results)

savings_per_patient_df


,Reduction Rate,Savings per Patient ($)
0,10%,295.75998
1,20%,591.51996
2,30%,887.27994


In [ ]:
# Export main dataset
df.to_csv("hospital_cleaned.csv", index=False)

# Export ROI data
roi_df.to_csv("roi_data.csv", index=False)

# Export savings per patient data
savings_per_patient_df.to_csv("savings_per_patient.csv", index=False)
